# Presenting and evaluating benchmark models

In [ ]:
# Basic imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import os

# Evaluation scripts
from CompetitionEvaluation import load_data, structure_data, calculate_metrics
 

In [ ]:
# Where to find files
username = os.getlogin()
Mydropbox = f'/Users/{username}/Dropbox (ViEWS)/ViEWS/'
overleafpath = f'/Users/{username}/Dropbox (ViEWS)/Apps/Overleaf/ViEWS predicting fatalities/Tables/'

print('Dropbox path set to',Mydropbox)
print('Overleaf path set to',overleafpath)

filepath = Mydropbox + 'Prediction_competition_2023/' 


## Reading in actuals

In [ ]:
df_cm_actuals = pd.read_parquet(filepath + 'cm_actuals.parquet')
df_pgm_actuals = pd.read_parquet(filepath + 'pgm_actuals.parquet')
df_cm_actuals.tail(), df_pgm_actuals.head()


In [ ]:
df_pgm_actuals.describe(percentiles=[.25,.50,.75,.90,.95,.99,.992,.995])

## Reading in benchmark prediction models: 

Two models per level:

1. cm model, based on ensemble
2. cm model, based on historical values 
3. pgm model, based on ensemble
4. pgm model, based on historical values

Ëach of these have predictions for each of four years; 2019, 2020, 2021, and 2022. The four years are collected in lists of dictionaries including dataframes and some metadata, one for each of the models above

In [ ]:
df_cm_actuals.query('country_id == 1')
df_cm_actuals.loc[445:468]

In [ ]:
bm_cm_ensemble = []
bm_cm_historical_values = []
bm_pgm_ensemble = []
bm_pgm_historical_values = []

for year in [2019, 2020, 2021]:
    print(year)
    first_month = (year - 1980)*12 + 1
    cm_e = {
        'year': year,
        'first_month': first_month,
        'name': 'cm_ensemble',
        'df_full': pd.read_parquet(filepath + 'bm_cm_ensemble_' + str(year) + '.parquet'),
        'df_agg': pd.read_parquet(filepath + 'bm_cm_ensemble_agg' + str(year) + '.parquet'),
        'actuals': df_cm_actuals.loc[first_month: first_month + 12 - 1],
    }
    bm_cm_ensemble.append(cm_e)
    cm_hv = {
        'year': year,
        'first_month': first_month,
        'name': 'cm_historical_values',
        'df_full': pd.read_parquet(filepath + 'bm_cm_historical_values_' + str(year) + '.parquet'),
        'df_agg': pd.read_parquet(filepath + 'bm_cm_historical_values_agg' + str(year) + '.parquet'),
        'actuals': df_cm_actuals.loc[first_month: first_month + 12 - 1],
    }
    bm_cm_historical_values.append(cm_hv)
    pgm_e = {
        'year': year,
        'first_month': first_month,
        'name': 'pgm_ensemble',
        'df_full': pd.read_parquet(filepath + 'bm_pgm_ensemble_' + str(year) + '.parquet'),
        'df_agg': pd.read_parquet(filepath + 'bm_pgm_ensemble_agg' + str(year) + '.parquet'),
        'actuals': df_pgm_actuals.loc[first_month: first_month + 12 - 1],
    }
    bm_pgm_ensemble.append(pgm_e)
    pgm_hv = {
        'year': year,
        'first_month': first_month,
        'name': 'pgm_historical_values',
        'df_full': pd.read_parquet(filepath + 'bm_pgm_historical_values_' + str(year) + '.parquet'),
        'df_agg': pd.read_parquet(filepath + 'bm_pgm_historical_values_agg' + str(year) + '.parquet'),
        'actuals': df_pgm_actuals.loc[first_month: first_month + 12 - 1],
    }
    bm_pgm_historical_values.append(pgm_hv)

In [ ]:
item['df_full'], item['name']

In [ ]:
# Restructuring, evaluating:
for model_list in [bm_cm_ensemble]:
    for item in model_list:
        print(item['name'], item['year'])
        
        item['observed'], item['predictions'] = structure_data(item['df_full'], item['actuals']) # structure data as xarrays that the xskillscore.crps_ensemble wants


        

In [ ]:
        

#observed, predictions = load_data(args.o, args.p) # read parquet files to pandas
observed, predictions = structure_data(df_pgm_actuals, df_bm_pgm_historical_values) # structure data as xarrays that the xskillscore.crps_ensemble wants
metrics = calculate_metrics(observed, predictions) # calculates crps.

In [ ]:
df_bm_cm_ensemble = pd.read_parquet(filepath + 'cm_benchmark_ensemble_550.parquet')
df_bm_cm_ensemble.describe()

In [ ]:
df_bm_cm_ensemble.head()

In [ ]:
df_bm_pgm_historical_values = pd.read_parquet(filepath + 'pgm_benchmark_historical_values_step_3.parquet')
df_bm_pgm_historical_values.describe()

In [ ]:
#observed, predictions = load_data(args.o, args.p) # read parquet files to pandas
observed, predictions = structure_data(df_cm_actuals, df_bm_cm_ensemble) # structure data as xarrays that the xskillscore.crps_ensemble wants
metrics = calculate_metrics(observed, predictions) # calculates crps.

In [ ]:
metrics

In [ ]:
# Read in for all 12 steps
from datetime import datetime
print("Cell started to run:", datetime.now())

df_pgm_hv = []
for step in range(3,14+1):
    df = pd.read_parquet(filepath + 'pgm_benchmark_historical_values_step_' + str(step) + '.parquet')
    print(step, df.describe())
    df_pgm_hv.append(df)
    
print("Cell run ended:", datetime.now())

In [ ]:
print("Cell started to run:", datetime.now())
i = 3
for df in df_pgm_hv:
    print('step',i,datetime.now())
    observed, predictions = structure_data(df_pgm_actuals, df) # structure data as xarrays that the xskillscore.crps_ensemble wants
    metrics = calculate_metrics(observed, predictions) # calculates crps.
    print(metrics)
    i=i+1
print("Cell run ended:", datetime.now())



# Read in the sc-type prediction files


In [ ]:
df_bm_pgm_ensemble2022 = pd.read_parquet(filepath + 'bm_pgm_ensemble_2022.parquet')
df_pgm_actuals_2022 = df_pgm_actuals.loc[505:516]
df_bm_pgm_ensemble2022.tail()

In [ ]:

observed, predictions = structure_data(df_pgm_actuals_2022, df_bm_pgm_ensemble2022) # structure data as xarrays that the xskillscore.crps_ensemble wants
metrics = calculate_metrics(observed, predictions) # calculates crps.
metrics

# Creating samples based on point predictions

Assuming Poisson distributions

In [ ]:
cm_ensemble_aggregated = pd.read_parquet(filepath + 'cm_benchmark_ensemble_550_aggregated.parquet')

print(cm_ensemble_aggregated.describe())
print(cm_ensemble_aggregated.head())

In [ ]:
# Strip down to a year of sc predictions:
df_cm_ensemble = []
for step in range(3,14+1):
    df = cm_ensemble_aggregated['mean_log_prediction'].loc[442+step]
    df = pd.DataFrame(df[df.index.get_level_values('step').isin([step])])
    df['prediction'] = np.expm1(df['mean_log_prediction'])
    df_cm_ensemble.append(df)

df_cm_ensemble_stripped = pd.concat(df_cm_ensemble)
print(df_cm_ensemble_stripped.describe())
print(df_cm_ensemble_stripped.tail(40))


In [ ]:
test = np.